# Case Study 2 - Predicting Hospital Readmittance

__Team Members:__ Amber Clark, Andrew Leppla, Jorge Olmos, Paritosh Rai

# Content
* [Business Understanding](#business-understanding)
    - [Abstract](#abstract)
    - [Introduction](#introduction)
    - [Methods](#methods)
    - [Results](#results)
* [Data Evaluation](#data-evaluation)
    - [Loading Data](#loading-data) 
    - [Data Summary](#data-summary)
    - [Missing Values](#missing-values)
    - [Feature Removal](#feature-removal)
    - [Exploratory Data Analysis (EDA)](#eda)
    - [Assumptions](#assumptions)
* [Model Preparations](#model-preparations)
    - [Sampling & Scaling Data](#sampling-scaling-data)
    - [Proposed Method](#proposed-metrics)
    - [Evaluation Metrics](#evaluation-metrics)
    - [Feature Selection](#feature-selection)
* [Model Building & Evaluations](#model-building)
    - [Sampling Methodology](#sampling-methodology)
    - [Model](#model)
    - [Performance Analysis](#performance-analysis)
* [Model Interpretability & Explainability](#model-explanation)
    - [Examining Feature Importance](#examining-feature-importance)
* [Conclusion](#conclusion)
    - [Final Model Proposal](#final-model-proposal)
    - [Future Considerations and Model Enhancements](#model-enhancements)
    - [Alternative Modeling Approaches](#alternative-modeling-approaches)

# Business Understanding & Executive Summary <a id='business-understanding'/>

### Objective<a id='scope'/>
This case study involves analyzing a ten-year study tracking diabetic patient readmission into hospitals. The goal of the analysis is to predict when diabetic patients are likely to be readmitted to a hospital based on the available data and determine if any of the provided factors are particularly indicative of a high chance of readmission.

### Introduction <a id='introduction'/>
Diabetes is a common disease that affects the body's ability to regulate blood sugar levels naturally. According to the American Heart Association [1], there are two main types of diabetes which involve issues with insulin. This hormone regulates how cells absorb glucose from the bloodstream. Type 1 diabetes describes the chronic form that is usually identified at a young age in which the body is unable to produce sufficient insulin. Type 2 diabetes, the most common condition, can arise later in life and occurs when the body develops an "insulin resistance" or its insulin production begins to diminish.
Complications involving the effects of diabetes on the heart and circulatory system can lead to dire conditions that require hospitalization. Often patients are released from the hospital but may have to be readmitted soon after with recurring issues, causing additional strain on both the patients' livelihoods and the efficiency of the hospital. The data provided for this case study is requisitioned from a ten-year study on hospital readmissions of diabetic patients. This analysis aims to predict when diabetic patients are likely to be readmitted to the hospital after a visit given the provided information and if that readmission will occur in less than 30 days. Based on a predictive model, the most influential indicators of probable readmissions can be identified so that medical professionals can make better judgments on whether a diabetic patient is ready to be released. Since diabetes is estimated to affect 463 million people worldwide [2], any improvement in this regard would have far-reaching significant benefits.

### EDA <a id='eda'/>
Two data CSV files are used for the analysis, "diabetic_data.csv" and "IDs_maping.csv". The “diabetic_data” dataset is based on the multi-year study of diabetes patients, and each row represents an individual hospital admission. The patients' information is systematically collected from the point of entry to that of discharge. It can be broadly classified into a few categories:
Personal information
- The admission situations and conditions
- The laboratory tests conducted
- The physician's diagnosis
- The treatments and medications
- The discharge conditions

This dataset contains 50 features and 101,766 rows, including the response variable readmission information. In the diabetic data file, the team identified missing data. Following are a few observations:
- The response variable is a bit unbalanced 89:11 split. 
- The imbalance was also observed in the male and female samples.
- Collected data did show more samples from the Caucasian race. However, the ratio of the patient taking diabetic medication vs. total patients within the race is very similar (range from 74% to 80%).  Hospitals are typically required by state law to collect demographic information like race and ethnicity.  The 1964 Civil Rights Act allows hospitals to collect this data to ensure no discrimination in care.
- Pair plot distribution of continuous variables did not show any obvious correlation among continuous variables.

The team looked at data in the "ID_mapping.csv" file. This file contains the ID for multiple categorical variables and its brief description. The team found some missing values like NULL, Not Available, unknown/Invalid, and Not Mapped during the data exploration of the ID_Mapping dataset. NaN replaced these missing values in the ID_Mapping dataset. Then Id_Mapping information was merged with the "diabetic_data".

### Methods <a id='methods'/>
The objective of our analysis is to classify diabetes patients who have the highest risk of being re-admitted to the hospital within 30 days using logistic regression modeling. L2 regularization was used to prevent overfitting and maintain model performance.  The team observed that L2 is faster and more efficient to run than L1 with this dataset that had a large number of dummy-coded variables.

The metrics used to evaluate model performance were precision and recall.  Precision and recall are useful metrics when the classes are imbalanced.  These metrics are defined as follows:    
- Recall = TP/(TP+FN)
- Precision = TP/(TP+ FP)

where TP = True Positives, FN = False Negatives, and FP = False Positives

In the best interest of the patient, the model would maximize recall such that hospital readmission is prevented at all costs. However, the resulting lower precision (higher false positive rate) would be costly.  It could lead to a shortage of beds or staff in the hospital, keeping more patients that are fine and that wouldn't be readmitted if dishcarged.  To balance both precision and recall, the team used the F1 score.  The F1 score combines precision and recall into a single metric by taking their harmonic mean:

F1 = 2 * Recall * Precision / (Recall + Precision)   

By maximizing the F1 score, the model balances both recall and precision for the unbalanced response.   

### Train and Test Data Split <a id='Train and Test Data Split'/>
The response variable is a somewhat unbalanced with 11% readmitted within 30 days and 89% not readmitted within 30 days. Stratified data splitting was used to ensure that these two clases are evenly split between the training and test sets.  A 70/30 split was used between the training and test sets.  Stratified data splitting  Dataset was split in training and test set before imputation. The continuous variables were scaled to ensure high magnitude variables were not influencing the outcome more than low magnitude variables.  

## Missing Values <a id='missing-values'>
No missing values were observed in any continuous variables. All the missing values were observed in just 9 of the 42 categorial features (see table below).  There were also missing values observed in Id_mapping files as mentioned in the EDA section above.  
    
The team investigated whether these missing values have any internal correlation or dependency. A heatmap did not show any obvious overlap or correlation with the missing values.
The diagnosis codes in columns diag_1, diag_2, and diag_3 reference Internation Classification of Diseases (ICD) codes. The data hierarchy of these codes was leveraged to reduce the number of category levels from 700-800 each to just 19 each.  This significantly reduced the number of dummy variables and made imputation much simpler.  See the code below for more details.    
https://www.aapc.com/codes/icd9-codes-range/
    
Following are the variables with the missing values and the methodology team used to mitigate the missing values:

| Missing Variable                 | # of missing values | Mitigation Methodology                                     |
|----------------------------------|---------------------|------------------------------------------------------------|
| weight                           | 98,569              | ~97% of data was missing so variable was dropped.          |
| race                             | 2,273               | Impute by Mode                                             |
| payer_code                       | 40,256              | Impute by Mode                                             |
| medical Specialty                | 49,949              | Group by admission_type_id_new_mapping then Impute by Mode |
| disharge_disposition_new_mapping | 4,680               | Impute by Mode                                             |
| admission_source_new_mapping     | 7,067               | Impute by Mode                                             |
| diag_1                           | 21                  | Recategorize and Impute by Mode                            |
| diag_2                           | 358                 | Recategorize and Impute by Mode                            |
| diag_3                           | 2,423               | Recategorize and Impute by Mode                            |

The race variable is identified as a potential ethical concern and a sensitive discussion topic, and the team decided to fill the missing values of race by the mode, Caucasian. The occurrence of missing information for race is relatively low, so there shouldn't be much concern for inadvertently creating any artificial trends in the model.

KNN Imputation Methodology:  Team also looked at KNN imputation to address missing values as there was no perfect correlation observed among missing variables. The team found that KNN imputation was not appropriate for the number of categorical variables in the dataset. The team decided to drop the KNN imputation and follow the imputation approach listed above.
 
### Other Data Cleanup <a id='Other Data Cleanup'/>
The team reviewed the data in detail after addressing the missing values. Features that are not meaningful such as encounter_id, patient_nbr, examide, and citoglipton are removed from the analysis. encounter_id and patient_nbr are id codes and will not add value to the model. examide, and citoglipton variables have all zeros.

### Logistic Regression Modeling <a id='Logistic Regression Modeling'/>
Logistic regression is a classification algorithm. It is used to predict a binary outcome based on the set of independent variables. Logistic regression is used for the analysis when working with the binary target variable, i.e., dichotomous, or categorical in nature; in other words, if it fits into one of two categories (such as “yes” or “no”, “pass” or “fail”, 0 or 1). The response variable used for this analysis also has dichotomous data, so the team decided to use Logistic Regression to analyze the diabatic data. 

Baseline Model:
There were no missing values in the 8 continuous variables, so the team built the baseline model using continuous variables only with Logistic Regression.  The team utilized this base model to see how the data looked without dummy coding or imputation, as well as assessing the value of adding more complex multi-level categorical variables.

The second Logistic Regression model was made using all the variables. This dataset was built after imputing any missing categorical values. The Logistic Regression model was built using L2 (Ridge) regularization, regularization penalty (C) of 1, and weighting the log loss by the target class (class_weight = "balanced"). The team chose balanced class_weight to deal with the imbalance of the target variable. This option balanced how the model was predicting negative and positive classes.
    
The accuracy of the model was 68%.
 
### Results <a id='results'/>


### References <a id='References'/>
[1] American Heart Association. What is Diabetes? https://www.heart.org/en/health-topics/diabetes/about-diabetes

[2] "IDF DIABETES ATLAS Ninth Edition 2019" (PDF). www.diabetesatlas.org. Retrieved 18 May 2020.
[3] What is Logistic Regression? A Beginner's Guide [2022] (careerfoundry.com)

 

# Data Evaluation <a id='data-evaluation'>
    

Summarize data being used?

Are there missing values?

Which variables are needed and which are not?

What assumptions or conclusions are you drawing about your data?

In [1]:
# standard libraries
import pandas as pd
import numpy as np
import os
from IPython.display import Image

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import pyplot
%matplotlib inline
from tabulate import tabulate

# data pre-processing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import StratifiedKFold
#from sklearn.model_selection import StratifiedGroupKFold
from sklearn.impute import SimpleImputer

# prediction models
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
#from kneed import KneeLocator
from scipy import stats

# metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import precision_recall_curve as pr_curve
from sklearn.metrics import plot_precision_recall_curve as plot_pr_curve
from sklearn.metrics import auc

# import warnings filter
'''import warnings
warnings.filterwarnings('ignore')
from warnings import simplefilter 
simplefilter(action='ignore', category=FutureWarning)'''

"import warnings\nwarnings.filterwarnings('ignore')\nfrom warnings import simplefilter \nsimplefilter(action='ignore', category=FutureWarning)"

## Loading Data <a id='loading-data'>

## Loading Id Mappings

### Admission Type

In [2]:
url = 'https://raw.githubusercontent.com/olmosjorge28/QTW-SPRING-2022/main/dataset_diabetes/IDs_mapping.csv'
admission_type_mapping = pd.read_csv(url, nrows=8, index_col=0)
admission_type_mapping

,description
admission_type_id,
1,Emergency
2,Urgent
3,Elective
4,Newborn
5,Not Available
6,NaN
7,Trauma Center
8,Not Mapped


### Discharge Disposition

In [3]:
discharge_disposition_mapping = pd.read_csv(url, nrows=30,skiprows=10, index_col=0 )
discharge_disposition_mapping 

,description
discharge_disposition_id,
1,Discharged to home
2,Discharged/transferred to another short term h...
3,Discharged/transferred to SNF
4,Discharged/transferred to ICF
5,Discharged/transferred to another type of inpa...
6,Discharged/transferred to home with home healt...
7,Left AMA
8,Discharged/transferred to home under care of H...
9,Admitted as an inpatient to this hospital


### Admission Source Mapping

In [4]:
admission_source_mapping = pd.read_csv(url,skiprows=42, index_col=0 )
admission_source_mapping

,description
admission_source_id,
1,Physician Referral
2,Clinic Referral
3,HMO Referral
4,Transfer from a hospital
5,Transfer from a Skilled Nursing Facility (SNF)
6,Transfer from another health care facility
7,Emergency Room
8,Court/Law Enforcement
9,Not Available


## Diabetes Dataset

In [5]:
url = 'https://raw.githubusercontent.com/olmosjorge28/QTW-SPRING-2022/main/dataset_diabetes/diabetic_data.csv'
df = pd.read_csv(url,na_values='?')

C:\Users\allep\anaconda3\envs\MyEnv3_7\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## Data Summary <a id='data-summary'>

| Feature name                | Type    | Description and values                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
|-----------------------------|---------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Encounter ID                | Numeric | Unique identifier of an encounter                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
| Patient number              | Numeric | Unique identifier of a patient                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| Race                        | Nominal | Values: Caucasian, Asian, African American, Hispanic, and other                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
| Gender                      | Nominal | Values: male, female, and unknown/invalid                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| Age                         | Nominal | Grouped in 10-year intervals: 0, 10), 10, 20), …, 90, 100)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| Weight                      | Numeric | Weight in pounds.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
| Admission type              | Nominal | Integer identifier corresponding to 9 distinct values, for example, emergency, urgent, elective, newborn, and not available                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
| Discharge disposition       | Nominal | Integer identifier corresponding to 29 distinct values, for example, discharged to home, expired, and not available                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
| Admission source            | Nominal | Integer identifier corresponding to 21 distinct values, for example, physician referral, emergency room, and transfer from a hospital                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
| Time in hospital            | Numeric | Integer number of days between admission and discharge                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| Payer code                  | Nominal | Integer identifier corresponding to 23 distinct values, for example, Blue Cross/Blue Shield, Medicare, and self-pay                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
| Medical specialty           | Nominal | Integer identifier of a specialty of the admitting physician, corresponding to 84 distinct values, for example, cardiology, internal medicine, family/general practice, and surgeon                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
| Number of lab procedures    | Numeric | Number of lab tests performed during the encounter                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
| Number of procedures        | Numeric | Number of procedures (other than lab tests) performed during the encounter                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| Number of medications       | Numeric | Number of distinct generic names administered during the encounter                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
| Number of outpatient visits | Numeric | Number of outpatient visits of the patient in the year preceding the encounter                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| Number of emergency visits  | Numeric | Number of emergency visits of the patient in the year preceding the encounter                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| Number of inpatient visits  | Numeric | Number of inpatient visits of the patient in the year preceding the encounter                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| Diagnosis 1                 | Nominal | The primary diagnosis (coded as first three digits of ICD9); 848 distinct values                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
| Diagnosis 2                 | Nominal | Secondary diagnosis (coded as first three digits of ICD9); 923 distinct values                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| Diagnosis 3                 | Nominal | Additional secondary diagnosis (coded as first three digits of ICD9); 954 distinct values                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| Number of diagnoses         | Numeric | Number of diagnoses entered to the system                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| Glucose serum test result   | Nominal | Indicates the range of the result or if the test was not taken. Values: >200, >300, normal, and none if not measured                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| A1c test result             | Nominal | Indicates the range of the result or if the test was not taken. Values: “>8” if the result was greater than 8%, “>7” if the result was greater than 7% but less than 8%, “normal” if the result was less than 7%, and “none” if not measured.                                                                                                                                                                                                                                                                                                                                                                                                              |
|                             |         |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
| Change of medications       | Nominal | Indicates if there was a change in diabetic medications (either dosage or generic name). Values: “change” and “no change”                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| Diabetes medications        | Nominal | Indicates if there was any diabetic medication prescribed. Values: “yes” and “no”                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
| 24 features for medications | Nominal | For the generic names: metformin, repaglinide, nateglinide, chlorpropamide, glimepiride, acetohexamide, glipizide, glyburide, tolbutamide, pioglitazone, rosiglitazone, acarbose, miglitol, troglitazone, tolazamide, examide, sitagliptin, insulin, glyburide-metformin, glipizide-metformin, glimepiride-pioglitazone, metformin-rosiglitazone, and metformin-pioglitazone, the feature indicates whether the drug was prescribed or there was a change in the dosage. Values: “up” if the dosage was increased during the encounter, “down” if the dosage was decreased, “steady” if the dosage did not change, and “no” if the drug was not prescribed |
| Readmitted                  | Nominal | Days to inpatient readmission. Values: “<30” if the patient was readmitted in less than 30 days, “>30” if the patient was readmitted in more than 30 days, and “No” for no record of readmission.                                                                                                                                                                                                                                                                                                                                                                                                                                                          |

## Remapping Data

### Re-mapping data based on ID mappings

In [6]:
def remapIds(*new_mappings: tuple) -> dict:
    mapping_dict = dict()
    for mapping in new_mappings:
        mapping_dict[mapping[1]] =  mapping[0].map(mapping[2])
    return mapping_dict

def refactorMappingIds(inputDf: pd.DataFrame) -> pd.DataFrame:
    admission_type_id_new_mapping = {
        1: "emergency",
        2: "urgent",
        3: "elective",
        4: "newborn",
        5: float("NaN"),
        6: float("NaN"),
        7: "trauma-center",
        8: float("NaN")
    }
    
    disharge_disposition_new_mapping = {
        1: "discharged",
        2: "transfer",
        3: "transfer",
        4: "transfer",
        5: "transfer",
        6: "transfer",
        7: "ama",
        8: "transfer",
        9: "admitted",
        10: "transfer",
        11: "expired",
        12: "admitted",
        13: "hospice",
        14: "hospice",
        15: "transfer",
        16: "transfer",
        17: "transfer",
        18: float("NaN"),
        19: "expired",
        20: "expired",
        21: "expired",
        22: "transfer",
        23: "transfer",
        24: "transfer",
        25: float("NaN"),
        26: float("NaN"),
        27: "transfer",
        28: "transfer",
        29: "transfer",
    }


    admission_sourcing_new_mapping = {
        1: "referral",
        2: "referral",
        3: "referral",
        4: "transfer",
        5: "transfer",
        6: "transfer",
        7: "emergency",
        8: "law-enforcement",
        9: float("NaN"),
        10: "transfer",
        11: "normal-delivery",
        12: "other-delivery",
        13: "other-delivery",
        14: "other-delivery",
        15: float("NaN"),
        17: float("NaN"),
        18: "transfer",
        19: "transfer",
        20: float("NaN"),
        21: float("NaN"),
        22: "transfer",
        23: "normal-delivery",
        24: "normal-delivery",
        25: "transfer",
        26: "transfer"
    }
    df = inputDf.copy()
    mapping_tuples = [
        (df['discharge_disposition_id'],'disharge_disposition_new_mapping', disharge_disposition_new_mapping),
        (df['admission_source_id'],'admission_source_new_mapping', admission_sourcing_new_mapping),
        (df['admission_type_id'], 'admission_type_id_new_mapping', admission_type_id_new_mapping)
    ]
    remappings = remapIds(*mapping_tuples)
    for newMappingKey in remappings:
        df[newMappingKey] = remappings[newMappingKey]
    return df

### Remapping Diag(nosis) Codes based on International Classification of Diseases (ICD) codes
https://www.aapc.com/codes/icd9-codes-range/

In [7]:
def getDiagCategory(input: float) -> str:
    val: str
    if input < 1:
        val = float("NaN")
    elif input < 140:
        val = 'A'
    elif input < 240:
        val = 'B'
    elif input < 280:
        val = 'C'
    elif input < 290:
        val = 'D'
    elif input < 320:
        val = 'E'
    elif input < 390:
        val = 'F'
    elif input < 460:
        val = 'G'
    elif input < 520:
        val = 'H'
    elif input < 580:
        val = 'I'
    elif input < 630:
        val = 'J'
    elif input < 680:
        val = 'K'
    elif input < 710:
        val = 'L'
    elif input < 740:
        val = 'M'
    elif input < 760:
        val = 'N'
    elif input < 780:
        val = 'O'
    elif input < 800:
        val = 'P'
    elif input < 1000:
        val = 'Q'
    elif input < 2000:
        val = 'R'
    elif input < 3000:
        val = 'S'
    else:
        val = 'Z'
    return val
    

def categorizeDiag(diag: pd.Series) -> pd.Series:
    df = diag.copy()
    df.fillna(0,inplace=True)
    df.mask(df.str.startswith('V', na=False),1000, inplace=True)
    df.mask(df.str.startswith('E', na=False),2000, inplace=True)
    df = pd.to_numeric(df)
    df = df.map(getDiagCategory)
    return df


def recategorizeDiags(inputDf: pd.DataFrame) -> pd.DataFrame:
    df = inputDf.copy()
    df['diag_1_categorized'] = categorizeDiag(df['diag_1'])
    df['diag_2_categorized'] = categorizeDiag(df['diag_2'])
    df['diag_3_categorized'] = categorizeDiag(df['diag_3'])
    return df


def addDiag(*diags):
    df = None
    for diag in diags:
        if (df is None):
            df = diag.notna().astype(int)
        else:
            df = df + diag.notna().astype(int)
    return df

In [8]:
def recategorizeData(inputDf: pd.DataFrame) -> pd.DataFrame:
    df = inputDf.copy()
    df = refactorMappingIds(df)
    df = recategorizeDiags(df)
    df['total_diag'] = addDiag(df['diag_1'],df['diag_2'],df['diag_3'])
    return df

In [9]:
df_recategorized = recategorizeData(df)

### Recoding Dependant Variable

In [10]:
def recode_variable(series: pd.Series, condition):
    s_copy = series.copy()
    s_copy = s_copy==condition
    s_copy = s_copy.astype(int)
    return s_copy

In [11]:
df_recategorized['readmitted'] = recode_variable(df_recategorized['readmitted'], '<30')

## Missing Values

In [12]:
df_recategorized.isna().sum()

encounter_id                            0
patient_nbr                             0
race                                 2273
gender                                  0
age                                     0
weight                              98569
admission_type_id                       0
discharge_disposition_id                0
admission_source_id                     0
time_in_hospital                        0
payer_code                          40256
medical_specialty                   49949
num_lab_procedures                      0
num_procedures                          0
num_medications                         0
number_outpatient                       0
number_emergency                        0
number_inpatient                        0
diag_1                                 21
diag_2                                358
diag_3                               1423
number_diagnoses                        0
max_glu_serum                           0
A1Cresult                         

## Feature Removal <a id='feature-removal'>

patient_nbr <br>
encounter_id <br>
examide <br>
citoglipton <br>
weight <br>

In [13]:
df_recategorized = df_recategorized.drop(['patient_nbr','encounter_id', 'examide', 'weight','citoglipton'], 1)

C:\Users\allep\anaconda3\envs\MyEnv3_7\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


discharge_disposition_id
admission_source_id
admission_type_id
diag_1
diag_2
diag_3

In [14]:
df_recategorized = df_recategorized.drop(['discharge_disposition_id','admission_source_id','admission_type_id','diag_1',
                'diag_2','diag_3'], 1)

C:\Users\allep\anaconda3\envs\MyEnv3_7\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


## Exploratory Data Analysis (EDA) <a id='eda'>

### 

### Feature Collinearity <a id='feature-collinearity'>


### Feature Outliers 
 

## Assumptions <a id='assumptions'>

# Model Preparations <a id='model-preparations'/>

What methods did you use (or not) to solve the problem?

Why are the methods you chose appropriate given the business objective?

How did you decide your approach was useful?  If more than one method, which one was better or why are each better or not?

What evaluation smetrics are most useful given the problem is a binary classification (ex. accuracy, f1-score, precision, recall AUC, etc)?



## Split dependant and Independant Variables

In [15]:
def split_dependant_and_independant_variales(df: pd.DataFrame, y_var: str):
    X = df.copy()
    y = X[y_var]
    X = X.drop([y_var], axis=1)
    return X, y

In [16]:
X, y = split_dependant_and_independant_variales(df_recategorized, 'readmitted')

## Data Imputation

In [17]:
def fitImputers(x_train: pd.DataFrame): 
    df = x_train.copy()
    imputer = SimpleImputer(missing_values = np.nan,
                        strategy ='most_frequent')
    columns_of_interest = ['disharge_disposition_new_mapping', 'admission_source_new_mapping', 'admission_type_id_new_mapping', 'diag_1_categorized', 'diag_2_categorized', 'diag_3_categorized', 'race', 'payer_code']
    imputer.fit(df[columns_of_interest])
    
    grouped_modes = df.groupby(['admission_type_id_new_mapping'])['medical_specialty'].agg(pd.Series.mode)
    grouped_modes['trauma-center'] = grouped_modes['urgent']
    
    def customApply(input):
        if(pd.isnull(input[1])):
            input[1] = grouped_modes[input[0]]
        return input
    return imputer, customApply

In [18]:
def imputeData(x_train: pd.DataFrame, x_test: pd.DataFrame):
    imputed_train = x_train.copy()
    imputed_test = x_test.copy()

    imputer, customApply = fitImputers(x_train)
    columns_of_interest = ['disharge_disposition_new_mapping', 'admission_source_new_mapping', 'admission_type_id_new_mapping', 'diag_1_categorized', 'diag_2_categorized', 'diag_3_categorized', 'race', 'payer_code']
    imputed_train[columns_of_interest] = imputer.transform(imputed_train[columns_of_interest])
    imputed_test[columns_of_interest] = imputer.transform(imputed_test[columns_of_interest])
    
    twoColumnsTrain = imputed_train[['admission_type_id_new_mapping','medical_specialty']]
    twoColumnsTest = imputed_test[['admission_type_id_new_mapping','medical_specialty']]
    
    twoColumnsTrain.apply(customApply, axis=1);
    twoColumnsTest.apply(customApply, axis=1);
    
    
    imputed_train['medical_specialty'] = twoColumnsTrain['medical_specialty']
    imputed_test['medical_specialty'] = twoColumnsTest['medical_specialty']
    
    imputed_train.to_csv('diabetic_data_train_imputed.csv')
    imputed_train = pd.read_csv('diabetic_data_train_imputed.csv')
    
    
    imputed_test.to_csv('diabetic_data_test_imputed.csv')
    imputed_test = pd.read_csv('diabetic_data_test_imputed.csv')
    
    imputed_train = imputed_train.drop(['Unnamed: 0'], 1)
    imputed_test = imputed_test.drop(['Unnamed: 0'], 1)
    
    return imputed_train, imputed_test

## Sampling & Scaling Data <a id='sampling-scaling-data' />

In [19]:
def shuffle_split(X, y, test_size, random_state):
    stratified_shuffle_split = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=random_state)
    for train_index, test_index in stratified_shuffle_split.split(X, y):
        X_train, X_test = X.iloc[train_index,:], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
    return X_train, X_test, y_train, y_test

def scale_data(X_train, X_test):
    scl = StandardScaler()
    scl.fit(X_train)
    X_train_scaled = pd.DataFrame(scl.transform(X_train), columns = X_train.columns.values, index = X_train.index)
    X_test_scaled = pd.DataFrame(scl.transform(X_test), columns = X_train.columns.values, index = X_test.index )
    return X_train_scaled, X_test_scaled


def encode_data(X_train, X_test):
    enc = OneHotEncoder(handle_unknown = 'ignore')
    enc.fit(X_train)
    X_train_encoded = pd.DataFrame( enc.transform(X_train).toarray(), 
                                   columns = enc.get_feature_names(X_train.columns), index = X_train.index)
    X_test_encoded = pd.DataFrame(enc.transform(X_test).toarray(), 
                                  columns = enc.get_feature_names(X_test.columns), index = X_test.index)
    return X_train_encoded, X_test_encoded

def scale_and_encode_data(x_train, x_test):
    cont_vars = x_train._get_numeric_data().columns
    X_cont_train_scaled, X_cont_test_scaled = scale_data(x_train[cont_vars], x_test[cont_vars])
    X_train_encoded, X_test_encoded = encode_data(x_train.drop(cont_vars, 1), x_test.drop(cont_vars, 1))
    return pd.concat([X_cont_train_scaled, X_train_encoded], axis=1), pd.concat([X_cont_test_scaled, X_test_encoded], axis=1)

## Prepare for Modeling

In [20]:
def model_preparation(X, y, test_split, random):
    X_train, X_test, y_train, y_test = shuffle_split(X, y, test_size = test_split, random_state= random)
    X_train, X_test = scale_and_encode_data(*imputeData(X_train, X_test))
    return X_train, X_test, y_train, y_test

In [21]:
X_train, X_test, y_train, y_test = model_preparation(X,y,0.3,1234)

C:\Users\allep\anaconda3\envs\MyEnv3_7\lib\site-packages\ipykernel_launcher.py:27: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
C:\Users\allep\anaconda3\envs\MyEnv3_7\lib\site-packages\ipykernel_launcher.py:28: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
C:\Users\allep\anaconda3\envs\MyEnv3_7\lib\site-packages\ipykernel_launcher.py:28: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


## Need to add one hot encoding for categorical variables

## Proposed Method <a id='proposed-metrics' />

## Evaluation Metrics <a id='evaluation-metrics' />

### Baseline Model

## Feature Selection <a id='feature-selection' />

# Model Building & Evaluations <a id='model-building'/>

Primary task is buiding a logistic regression to predict hospital readmittances.

How did you handle missing values?

Specify your sampling methodology

Set up your models - highlights of any important parameters

Analysis of your models performance

## Model

In [22]:
def run_lgr(penalty, c, cv_number, x_train, x_test, y_train, y_test):
    lgr = LogisticRegression(penalty=penalty, C=c, solver='liblinear', class_weight='balanced')
    lgr.fit(x_train,y_train)
    y_hat = lgr.predict(x_test)
    return y_hat, y_test

In [23]:
 y_hat, y_test = run_lgr('l2', 0.5, 3,X_train,X_test, y_train, y_test)

In [24]:
confusion_matrix(y_test, y_hat)

array([[17783,  9340],
       [ 1458,  1949]], dtype=int64)

## Sampling Methodology <a id='sampling-methodology'/>

#### Per the code above we used a 70/30 train test sample split with 5 fold internal cross validation for training

## Model's Performance Analysis <a id='performance-analysis'/>

# Model Interpretability & Explainability <a id='model-explanation'>

Which variables were more important and why?

How did you come to the conclusion these variables were important how how should the audience interpret this?

## Examining Feature Importance <a id='examining-feature-importance'/>

# Conclusion <a id='conclusion'>

What are you proposing to the audience with your models and why?

How should your audience interpret your conclusion and whwere should they go moving forward on the topic?

What other approaches do you recommend exploring?

Bring it all home!

Need a conversation with stakeholders regarding:
- Balance of precision vs. recall.  What is the cost of a false positive vs. a false negative?  A high false positive rate could lead to a shortage of beds in the hospital by keeping patients that are fine and won't be readmitted in the next 30 days.  A high false negative rate will cause undue stress and possible harm to patients who should be kept in the hospital.
- Level of detail with diagnoses columns (diag 1-3).  We generalized these diagnoses such that there were fewer levels to model.  We recommend reviewing the top factor recoded diagnoses categories to see if any warrant a deeper dive


### Final Model Proposal <a id='final-model-proposal'/>

### Future Considerations and Model Enhancements <a id='model-enhancements'/>

### Alternative Modeling Approaches <a id='alternative-modeling-approaches'>